In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [ ]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

In [ ]:
print(mnist.train.labels[0].shape, mnist.train.images[0].shape)
print(mnist.train.images.shape, mnist.test.images.shape)
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32,[None, 10])
x_shape = tf.reshape(x,[-1,28,28,1])




In [ ]:
def newconl (input_data, input_channels, filters, filtershape, pool_shape):
    confilshape = [filtershape[0], filtershape[1], input_channels, filters]
    w = tf.Variable(tf.truncated_normal(confilshape, stddev= 0.03))
    b = tf.Variable(tf.truncated_normal([filters]))
    op = tf.nn.conv2d(input_data, w,[1,1,1,1], padding='SAME')
    op+=b
    op+= tf.nn.relu(op)
    psize = [1, pool_shape[0],pool_shape[1],1]
    s=[1,2,2,1]
    
    op = tf.nn.max_pool(op,ksize = psize, strides = s, padding='SAME')
    return op    
    
    
    
    

In [ ]:
#print(x_shape.shape)
l1 = newconl(x_shape,1,32,[5,5],[2,2])
l2 = newconl(l1,32,64,[5,5],[2,2])
tf.shape(l2)

In [ ]:
f = tf.reshape(l2,[-1,7*7*64])
w1 = tf.Variable(tf.truncated_normal([7*7*64,1000], stddev=0.03))
b1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01))
d1 = tf.matmul(f,w1)+b1
d1 = tf.nn.relu(d1)
d1.shape




In [ ]:
w2 = tf.Variable(tf.truncated_normal([1000,10], stddev=0.03))
b2 = tf.Variable(tf.truncated_normal([10], stddev=0.01))
d2=  tf.matmul(d1,w2)+b2
y_ = tf.nn.softmax(d2)

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=d2,labels=y)
mean = tf.reduce_mean(loss)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.00001).minimize(mean)

In [ ]:
cpred = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
cpred = tf.cast(cpred,tf.float32)
acc  = tf.reduce_mean(cpred)

In [ ]:
sess = tf.Session()

In [ ]:
b = int(len(mnist.train.labels)/50)
b

In [ ]:
time = 10
intial = tf.global_variables_initializer()
sess.run(intial)


for i in range(time):
    print(i+1)
    cost =0
    for j in range(b):
        X, Y = mnist.train.next_batch(batch_size=50)
        _, c = sess.run([optimizer,loss],feed_dict={x:X,y:Y})
        cost += (c/b)
        
    accg = sess.run(acc , feed_dict={x:mnist.train.images, y:mnist.train.labels})
    print("cost=",cost, "accuracy=",accg)
    
    